In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2
import json
import pandas as pd
from pandas import read_csv
import json

In [3]:
# 1. Wir müssen unsere Daten laden 
# 2. Daten prozessieren -> Daten alle kleinschreiben, und dann in Vektor umwandeln 
# 3. One hot encoding -> stellt bestimmten Wert als 1 der 0 dar 

In [4]:
#lade einfache Daten und wandle in einen Dataframe um
with open('testEinfach1.json') as einfachJson:
    json_einfach_for_einfachData = json.load(einfachJson)

#das sind alle Events mit Element id von einfach
reduzed_json_einfach_for_einfachData = []

#das sind alle Events mit Element id von einfach
for index, item in enumerate(json_einfach_for_einfachData):
    elementId = item.get('elementId')
    if (elementId == 'inputCountry' or elementId == 'inputHouse' or elementId ==  'inputPet' or elementId ==  'inputHand' or elementId ==  'inputBundesland' or elementId == 'inputEasyQuestion' or (elementId == 'window' and item.get('siteName') == 'TestEinfach')):
        reduzed_json_einfach_for_einfachData.append(item)

In [5]:
with open('testEinfach1.json') as einfachJson:
    json_einfach_for_missing_keys = json.load(einfachJson)

#array mit allen Events die property key haben in TestEinfach
array_with_keys_from_einfach = []

#get all events with key 
for index, item in enumerate(json_einfach_for_missing_keys):
    if(item.get('key')):
        array_with_keys_from_einfach.append(item)

#array mit allen Events die jetzt richtig zugeteilt wurden zu TestSchwer
items_with_correct_keys = []

#get all events which are actually in TestSchwer
for index, item in enumerate(array_with_keys_from_einfach):
    elementId = item.get('elementId')
    if(elementId == 'inputHobby' or elementId =='inputCancelor' or elementId =='inputTemperature' or elementId =='inputDays' or elementId =='inputRivers' or elementId =='inputDifficultQuestion'):
        item['siteName'] = 'TestSchwer'
        items_with_correct_keys.append(item)


In [6]:
with open('testSchwer1.json') as schwerJson: 
    json_schwer = json.load(schwerJson)

#richtige schwer Events + array mit allen Events die jetzt richtig zugeteilt wurden zu TestSchwer
extended_json_schwer = json_schwer + items_with_correct_keys

#extended_json_schwer => alle Schwer Events 
#reduzed_json_einfach_for_einfachData => alle Einfach Events

In [7]:
# ohne 'key' Spalte
df_schwer = pd.DataFrame(extended_json_schwer)
df_schwer_without_key = df_schwer.drop(['key'], axis=1)
#6470 Events ohne scroll
df_schwer_without_scroll = df_schwer_without_key.loc[df_schwer_without_key["type"] !="scroll"]

df_einfach = pd.DataFrame(reduzed_json_einfach_for_einfachData)
df_einfach_without_key = df_einfach.drop(['key'], axis=1)
#1679 Events ohne scroll
df_einfach_without_scroll = df_einfach_without_key.loc[df_einfach_without_key["type"] !="scroll"]

In [8]:
sessionIdSchwer = df_schwer['session']
sessionIdEinfach = df_einfach['session']

df_einfach['is_duplicated'] = df_einfach.duplicated('session')
#86 ids auf Seite Einfach
setEinfach = set(sessionIdEinfach)
length = len(setEinfach)

#59 ids auf Seite Schwer
df_schwer['is_duplicated'] = df_schwer.duplicated('session')
setSchwer = set(sessionIdSchwer)
lengthSchwer = len(setSchwer)


In [9]:
in_schwer_but_not_in_einfach = setSchwer - setEinfach
in_einfach_but_not_in_schwer = setEinfach - setSchwer

array_einfach = list(in_einfach_but_not_in_schwer)
array_schwer = list(in_schwer_but_not_in_einfach)

# Anzahl Events | sessionId | Klasse 
# 20 | 95 | 0
# 30 | 113 | 0
# 10 | 113 | 1


In [10]:
#52 Teilnehmer bzw. Sessions
df_einfach_without_ids = df_einfach_without_scroll[~df_einfach_without_scroll['session'].isin(array_einfach)]
df_schwer_without_ids = df_schwer_without_scroll[~df_schwer_without_scroll['session'].isin(array_schwer)]

In [11]:
#remove columns
df_removed_columns_einfach = df_einfach_without_ids.drop(['timeStamp', 'type', 'date'], axis=1)
df_removed_columns_schwer = df_schwer_without_ids.drop(['timeStamp', 'type', 'date'], axis=1)

In [12]:
#replace values of columns 
df_new_siteName_einfach = df_removed_columns_einfach.replace('TestEinfach', 0)
df_new_siteName_schwer = df_removed_columns_schwer.replace('TestSchwer', 1)

arrayOfEvents = ['clickHere', 'buttonYes', 'nextButton', 'alibiClick']

df_removed_inputFields_schwer = df_new_siteName_schwer[~df_new_siteName_schwer['elementId'].isin(arrayOfEvents)]

In [29]:
from random import random
from secrets import choice

#df_merged_einfach_schwer = pd.concat([df_removed_inputFields_schwer, df_new_siteName_einfach])
array_of_ids_einfach = df_new_siteName_einfach['session'].value_counts().index.tolist()
array_of_ids_schwer = df_removed_inputFields_schwer['session'].value_counts().index.tolist()

random_numbers_einfach = np.random.randint(101,1000,size=len(array_of_ids_einfach))
random_numbers_schwer = np.random.choice([i for i in range(101,1000) if i not in random_numbers_einfach], size=len(array_of_ids_schwer))

[422 405 962 538 997 277 376 173 430 644 281 418 508 776 334 475 825 957
 194 828 890 811 892 837 366 889 385 761 316 164 579 574 866 218 598 919
 791 341 864 350 685 426 882 110 549 715 847 765 540 607 646 963]
[539 498 313 219 456 670 179 345 737 839 470 193 479 807 322 227 402 766
 332 950 709 674 498 339 128 477 505 698 697 560 142 955 261 127 601 137
 482 576 782 123 273 921 125 291 870 588 923 916 265 734 233 545]


In [34]:
df_replaced_session_einfach = df_new_siteName_einfach.replace(array_of_ids_einfach, random_numbers_einfach)
df_replaced_session_schwer = df_removed_inputFields_schwer.replace(array_of_ids_schwer, random_numbers_schwer)

pd.set_option("display.max_rows", None, "display.max_columns", None)

#df_replaced_session_einfach['session'].value_counts()
#df_replaced_session_schwer['session'].value_counts()


In [43]:
#create new Dataframe 
new_df_einfach = pd.DataFrame(df_replaced_session_einfach['session'].value_counts().reset_index().values, columns=["session", "eventCount"])
class_num_einfach = '0'
df_final_einfach = new_df_einfach.assign(class_num=class_num_einfach)
print(df_final_einfach)

new_df_schwer = pd.DataFrame(df_replaced_session_schwer['session'].value_counts().reset_index().values, columns=["session", "eventCount"])
class_num_schwer = '1'
df_final_schwer = new_df_schwer.assign(class_num=class_num_schwer)
print(df_final_schwer)



    session  eventCount class_num
0       498          68         0
1       539          50         0
2       219          42         0
3       313          42         0
4       456          38         0
5       670          37         0
6       345          32         0
7       179          32         0
8       737          32         0
9       839          31         0
10      193          30         0
11      479          30         0
12      470          30         0
13      807          28         0
14      322          28         0
15      227          27         0
16      950          26         0
17      402          26         0
18      332          26         0
19      709          26         0
20      766          26         0
21      674          25         0
22      339          25         0
23      261          24         0
24      560          24         0
25      697          24         0
26      505          24         0
27      698          24         0
28      477   

In [15]:
# #TestSchwer
# timestampsSchwer = [i["timeStamp"] for i in datasetSchwer["data"]]
# typesSchwer = [i['type'] for i in datasetSchwer['data']]

# df = pd.DataFrame({'timestamp': timestampsSchwer, 'types': typesSchwer})

# plt.bar(timestampsSchwer, typesSchwer)

In [16]:
# from pandas import json_normalize

# normalized_data = json_normalize(dataset['programs'])

# normalized_data.drop('_id.$oid', inplace=True, axis=1)


# normalized_data.head(10)




In [17]:
normalized_data.duplicated().sum


NameError: name 'normalized_data' is not defined

In [ ]:
normalized_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   clientId   347 non-null    object
 1   elementId  347 non-null    object
 2   timeStamp  347 non-null    object
 3   type       347 non-null    object
 4   siteName   347 non-null    object
 5   date       347 non-null    object
 6   clientX    133 non-null    object
 7   clientY    133 non-null    object
 8   browser    180 non-null    object
dtypes: object(9)
memory usage: 24.5+ KB
